In [ ]:
import json
import pickle
import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator
import torch.nn.functional as F
import spacy
import numpy as np

import random
import math
import time

from torchtext.data import TabularDataset

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!git clone https://github.com/corticai/open_book_qa.git

fatal: destination path 'open_book_qa' already exists and is not an empty directory.


In [ ]:
traindf = pd.read_csv('/content/open_book_qa/data/train/train.csv')

In [ ]:
traindf.head()

,id,context,question,answer_label_idx,label_A,label_B,label_C,label_D
0,7-980,the sun is the source of energy for physical c...,The sun is responsible for,3,puppies learning new tricks,children growing up and getting old,flowers wilting in a vase,"plants sprouting, blooming and wilting"
1,7-584,"as distance to an object increases , that obje...",When standing miles away from Mount Rushmore,3,the mountains seem very close,the mountains are boring,the mountains look the same as from up close,the mountains seem smaller than in photographs
2,7-870,digestion is when stomach acid breaks down food,When food is reduced in the stomach,2,the mind needs time to digest,take a second to digest what I said,nutrients are being deconstructed,reader's digest is a body of works
3,7-321,a star is made of gases,Stars are,2,warm lights that float,made out of nitrate,great balls of gas burning billions of miles away,lights in the sky
4,9-732,Galileo Galilei made improvements to the teles...,You can make a telescope with a,3,straw,Glass,Candle,mailing tube


In [ ]:
traindf['Answer'] = ""

In [ ]:
answer = []
for i in range(traindf.shape[0]):
  if traindf.iloc[i].answer_label_idx == 0:
    answer.append(traindf.iloc[i].label_A)
  elif traindf.iloc[i].answer_label_idx == 1:
    answer.append(traindf.iloc[i].label_B)
  elif traindf.iloc[i].answer_label_idx == 2:
    answer.append(traindf.iloc[i].label_C)
  elif traindf.iloc[i].answer_label_idx == 3:
    answer.append(traindf.iloc[i].label_D)

In [ ]:
traindf['Answer'] = answer

In [ ]:
traindf = traindf[['question','Answer']]

In [ ]:
traindf.head()

,question,Answer
0,The sun is responsible for,"plants sprouting, blooming and wilting"
1,When standing miles away from Mount Rushmore,the mountains seem smaller than in photographs
2,When food is reduced in the stomach,nutrients are being deconstructed
3,Stars are,great balls of gas burning billions of miles away
4,You can make a telescope with a,mailing tube


In [ ]:
if not os.path.exists('preprocessed_train.csv'):
    traindf.to_csv('preprocessed_train.csv', index = False)

In [ ]:
testdf = pd.read_csv('/content/open_book_qa/data/test/test.csv')
testdf['Answer'] = ""
answer = []
for i in range(testdf.shape[0]):
  if testdf.iloc[i].answer_label_idx == 0:
    answer.append(testdf.iloc[i].label_A)
  elif testdf.iloc[i].answer_label_idx == 1:
    answer.append(testdf.iloc[i].label_B)
  elif testdf.iloc[i].answer_label_idx == 2:
    answer.append(testdf.iloc[i].label_C)
  elif testdf.iloc[i].answer_label_idx == 3:
    answer.append(testdf.iloc[i].label_D)

testdf['Answer'] = answer
testdf = testdf[['question','Answer']]
testdf.head()

,question,Answer
0,A person wants to start saving money so that t...,quit eating lunch out
1,There is most likely going to be fog around:,a marsh
2,Predators eat,bunnies
3,Oak tree seeds are planted and a sidewalk is p...,parts may break the concrete
4,An electric car runs on electricity via,electrical conductors


In [ ]:
if not os.path.exists('preprocessed_valid.csv'):
    testdf.to_csv('preprocessed_valid.csv', index = False)

In [ ]:
spacy_en = spacy.load('en')

In [ ]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
QSTN = Field(tokenize=tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

ANS = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [ ]:
trainset = TabularDataset(path='preprocessed_train.csv', format='csv', skip_header=True,
                            fields=[('question', QSTN), ('Answer', ANS)])

In [ ]:
test_data = TabularDataset(path='preprocessed_valid.csv', format='csv', skip_header=True,
                            fields=[('question', QSTN), ('Answer', ANS)])

In [ ]:
train_data, valid_data = trainset.split(split_ratio = 0.7, random_state=random.getstate())  

In [ ]:
vars(train_data.examples[0])

{'Answer': ['it', 'underwent', 'a', 'tearing', 'process'],
 'question': ['if',
  'a',
  'book',
  'was',
  'intact',
  'on',
  'the',
  'desk',
  'in',
  'the',
  'morning',
  ',',
  'but',
  'in',
  'shreds',
  'in',
  'the',
  'afternoon',
  ',',
  'what',
  'happened',
  '?']}

In [ ]:
vars(valid_data.examples[0])

{'Answer': ['scars'],
 'question': ['when',
  'the',
  'mountain',
  'held',
  'the',
  'hound',
  "'s",
  'face',
  'to',
  'the',
  'fire',
  ',',
  'it',
  'caused']}

In [ ]:
QSTN.build_vocab(train_data, min_freq = 2)
ANS.build_vocab(train_data, min_freq = 2)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device, sort_key=lambda x: len(x.question), # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=False,)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [ ]:
INPUT_DIM = len(QSTN.vocab)
OUTPUT_DIM = len(ANS.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2309, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1420, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=1420, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,925,516 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = ANS.vocab.stoi[ANS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.question
        trg = batch.Answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        # print('Training Done')
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question
            # print(src)
            trg = batch.Answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 6
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 1s
	Train Loss: 5.784 | Train PPL: 324.970
	 Val. Loss: 4.094 |  Val. PPL:  59.965
Epoch: 02 | Time: 0m 1s
	Train Loss: 4.730 | Train PPL: 113.246
	 Val. Loss: 4.034 |  Val. PPL:  56.512
Epoch: 03 | Time: 0m 1s
	Train Loss: 4.536 | Train PPL:  93.347
	 Val. Loss: 4.052 |  Val. PPL:  57.484
Epoch: 04 | Time: 0m 1s
	Train Loss: 4.457 | Train PPL:  86.213
	 Val. Loss: 3.968 |  Val. PPL:  52.863
Epoch: 05 | Time: 0m 1s
	Train Loss: 4.402 | Train PPL:  81.648
	 Val. Loss: 3.958 |  Val. PPL:  52.341
Epoch: 06 | Time: 0m 1s
	Train Loss: 4.383 | Train PPL:  80.106
	 Val. Loss: 3.956 |  Val. PPL:  52.258


In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.090 | Test PPL:  59.715 |
